In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
from flags_pennants import find_flags_pennants_pips, find_flags_pennants_trendline, add_flag_signals
import vectorbtpro as vbt
vbt.settings.plotting["layout"]["template"] = "vbt_dark"


In [ ]:

# Load your data into a pandas DataFrame
df = pd.read_csv('BTCUSDT3600.csv')
order = 10
# Convert the 'date' column to a datetime object and set it as the index of the DataFrame
df['date'] = df['date'].astype('datetime64[s]')
df = df.set_index('date')

# Apply the natural logarithm to the 'close' column
df['log_close'] = np.log(df['close'])

# Identify flag patterns using the PIPs method
bull_flags, bear_flags, _, _ = find_flags_pennants_pips(df['log_close'], order)



In [ ]:
df = add_flag_signals(df, bull_flags, bear_flags)
df

In [ ]:
#Set flag_width NaN values to 0
df['flag_width'] = df['flag_width'].fillna(0)

# Set flag_width to an integer
df['flag_width'] = df['flag_width'].astype(int)


In [ ]:
df['flag_width'].describe()

In [ ]:
print

In [ ]:
# Print out the number of Bull Flags, Bear Flags, and total Flags
print(f'Number of Bull Flags: {df.bull_flag.sum()}')
print(f'Number of Bear Flags: {df.bear_flag.sum()}')

In [ ]:
# Simulate a portfolio using the vectorbt Portfolio class

pf = vbt.Portfolio.from_signals(
    df['close'],
    entries=df['bull_flag'],
    short_entries=df['bear_flag'],
    td_stop = df['flag_width']*.5,
    time_delta_format = 'Rows',
    leverage = 2,
)
print(pf.stats())

pf.plot(height=600, width=600).show_svg() 

# Now Run it on Dollar Bars

In [ ]:

# Load your data into a pandas DataFrame
df = vbt.BinanceData.load('/Users/ericervin/Documents/Coding/sigma/data/btc_90M_db_vbt.pkl').get() # This can be found in the sigma repo
order = 75
# Adjust a few column names for convenience
df['date'] = df['Open Time']
df = df.set_index('date')
df['close'] = df['Close']
# Apply the natural logarithm to the 'close' column
df['log_close'] = np.log(df['close'])

# Identify flag patterns using the PIPs method
bull_flags, bear_flags, _, _ = find_flags_pennants_pips(df['log_close'], order)

df = add_flag_signals(df, bull_flags, bear_flags)
#Set flag_width NaN values to 0
df['flag_width'] = df['flag_width'].fillna(0)

# Set flag_width to an integer
df['flag_width'] = df['flag_width'].astype(int)

# Simulate a portfolio using the vectorbt Portfolio class

pf = vbt.Portfolio.from_signals(
    df['close'],
    entries=df['bull_flag'],
    short_entries=df['bear_flag'],
    td_stop = df['flag_width']*1,
    time_delta_format = 'Rows',
    leverage = 2,
)
print(pf.stats())

pf.plot(height=600, width=600).show_svg() 